In [ ]:
import json
import pandas as pd
import tradingeconomics as te

In [ ]:
te.login()


In [ ]:
raw_data = te.getHistoricalData(country=['Bangladesh', 'China', 'Japan', 'Israel', 'India', 'South Korea'], indicator='Wages', initDate='2015-03-01', output_type='df')

In [ ]:
country_df = raw_data.loc[raw_data['Country'] == 'South Korea']
country_df = pd.DataFrame(country_df)

country_df.sort_values(by='DateTime', inplace=True, ascending=True)
average_wage = country_df.iloc[-1,]
average_wage

In [ ]:
def func_wage(country, ticker):
    raw_data = te.getHistoricalData(country=country, indicator='Wages', initDate='2015-03-01', output_type='df')    
    country_name = raw_data.loc[raw_data['Country'] == country]
    country_df = pd.DataFrame(country_name)

    country_df.sort_values(by='DateTime', inplace=True, ascending=True)
    average_wage = country_df.iloc[-1,]
    
    average_wage = pd.DataFrame(average_wage).T
    
    # Convert Currency
    average_wage['Value'] = average_wage['Value']/Convert_Curr(country, ticker)
    
    # Convert Yearly Freq
    average_wage = Convert_Yearly(country, average_wage)
    
    average_wage['Value'] = average_wage['Value'].values[0].round(1)
    
    return average_wage

In [ ]:
def truncate(number, decimals=0):
    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [ ]:
def Convert_Curr(country, ticker):
    currency = te.getCurrencyCross(cross = ticker, output_type = 'df')
    currency = currency.loc[currency['Country'] == country]
    output = currency['Close']
    output = output.values[0]
    # output = truncate(output, 2)
    return output

In [ ]:
#Convert yearly wage to monthly
def Convert_Yearly(country, average_wage):
    raw_data = te.getIndicatorData(country=country, indicators='wages', output_type='df')
    unit = raw_data['Unit'].values[0]
    unit = unit.split("/")
    unit = unit[1]
    
    if unit == 'Year':
        average_wage['Value'] = (average_wage['Value']/12)
    else:
        return average_wage
        
    average_wage['Value'] = average_wage['Value'].values[0].round(1)
    return average_wage

In [ ]:
with open('country.json', 'r') as f:
    data = json.load(f)
    
data = pd.DataFrame(data['data'])

In [ ]:
data

In [ ]:
length = len(data.index)

Average_Wage_in_Asia = pd.DataFrame()
for i in range(length):
    w = func_wage(data['name'].values[i], data['ticker'].values[i])
    Average_Wage_in_Asia = pd.concat([Average_Wage_in_Asia, w])

In [ ]:
Wage = Average_Wage_in_Asia
Wage

In [ ]:
value = []
value = Average_Wage_in_Asia['Value'].values
wage_in_float = pd.DataFrame(value).astype(float)

In [ ]:
Wage['Value'] = '$' + Wage['Value'].astype(str)
data['wage'] = Wage['Value'].values
data['wage_in_float'] = wage_in_float

In [ ]:
scale = {'Color': ['green', 'blue', 'orange', 'red'], 'Wage': ['above $2000', ' $1000 to $1,999', '$500 to $999', 'below $500']}  
scale = pd.DataFrame(scale)
scale

def color_range(wage):
    if wage < 500:
        color = "red"
    elif (wage >= 500) & (wage <= 999):
        color = "orange"
    elif (wage >= 1000) & (wage <= 1999):
        color = "blue"
    elif (wage>=2000):
        color = "green"
    
    return color

In [ ]:
import folium

m = folium.Map(location=[34.0479, 100.6197], zoom_start=3)

folium.LayerControl().add_to(m)

for i in range(len(data.index)):
    folium.Marker(location=[data['lat'].values[i], data['lon'].values[i]],
             icon = folium.DivIcon(
                 html="""
                 <div style='color: white;background-color:"""+color_range(data['wage_in_float'].values[i])+"""; width: 90px; padding: 2px; text-align: center'>
                     """+ data['name'].values[i] +" "+data['wage'].values[i]+"""
                 </div>
                 """,
                 class_name = "mapText2"
             )).add_to(m)

m